**CRNN Number Recognition**

Import needed libraries.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

**Dataset**

In [ ]:
class NumberDataset(Dataset):
    def __init__(self, img_dir, label_file, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        with open(label_file, 'r') as f:
            lines = f.readlines()
        self.samples = [line.strip().split(',') for line in lines]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(os.path.join(self.img_dir, img_path)).convert('L')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor([int(c) for c in label], dtype=torch.long)
        return image, label

**CRNN Model**

In [ ]:
class CRNN(nn.Module):
    def __init__(self, imgH, nc, nclass, nh):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(nc, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2,2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2,2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2,2)
        )
        self.rnn = nn.LSTM(256, nh, bidirectional=True, num_layers=2, batch_first=True)
        self.fc = nn.Linear(nh*2, nclass)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        x = x.permute(0, 3, 1, 2)  # [batch, width, channels, height]
        x = x.view(b, w, c*h)
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

**Hyperparameters & DataLoader**

In [ ]:
imgH = 32
nc = 1
nclass = 11  # 0-9 digits + blank for CTC
nh = 128

transform = transforms.Compose([
    transforms.Resize((imgH, 128)),
    transforms.ToTensor()
])

train_dataset = NumberDataset('data/train', 'data/train_labels.txt', transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = NumberDataset('data/val', 'data/val_labels.txt', transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

**Training Loop**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CRNN(imgH, nc, nclass, nh).to(device)
criterion = nn.CTCLoss(blank=10)
optimizer = optim.Adam(model.parameters(), lr=0.001)

def encode_labels(labels, max_len):
    targets = []
    lengths = []
    for label in labels:
        targets.extend(label.tolist())
        lengths.append(len(label))
    return torch.tensor(targets, dtype=torch.long), torch.tensor(lengths, dtype=torch.long)

for epoch in range(10):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        targets, lengths = encode_labels(labels, labels.size(1))
        targets = targets.to(device)
        lengths = lengths.to(device)
        output = model(images)  # [batch, width, nclass]
        output = output.log_softmax(2)
        input_lengths = torch.full(size=(output.size(0),), fill_value=output.size(1), dtype=torch.long)
        loss = criterion(output.permute(1,0,2), targets, input_lengths, lengths)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

**Evaluation**

In [ ]:
model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        output = model(images)
        output = output.softmax(2)
        pred = output.argmax(2)
        # Decode predictions as needed
        print("Predicted:", pred[0].cpu().numpy())
        print("True:", labels[0].cpu().numpy())
        break  # Show one batch

**Save model**

In [ ]:
torch.save(model.state_dict(), 'crnn_number_recognition.pth')